In [1]:
import json
import glob
from langchain_community.document_loaders import JSONLoader
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain.agents import create_agent
import time

In [2]:
FHIR_BUNDLE_JQ_SCHEMA = ".entry[]"
FHIR_BUNDLE_CONTENT_KEY = ".resource"

synthea_bundles = glob.glob("./fhir/*.json")

LLM = ChatOllama(model="llama3.2:1b") # using smaller model due to resource limitations on my old laptop 🙃

In [3]:
@tool
def get_fhir_data() -> list[str]:
    """Retrieves FHIR data for a set of Patients, including details about Procedures

    Returns:
        List of FHIR Resources as JSON
    """
    loaders = [
        JSONLoader(
            file_path=xpath,
            jq_schema=FHIR_BUNDLE_JQ_SCHEMA,
            content_key=FHIR_BUNDLE_CONTENT_KEY,
            is_content_key_jq_parsable=True,
            text_content=False
        )
        for xpath in synthea_bundles[:3] # limiting to just the first 3 for now
    ]

    docs = []
    for loader in loaders:
        docs.extend(loader.load())
    
    return docs


In [4]:
fhir_retrieval_agent = create_agent(
    model=LLM,
    tools=[get_fhir_data],
    system_prompt="You retrieve patient records and summarize based on the user query"
)

In [5]:
start_time = time.time()
response = fhir_retrieval_agent.invoke(
    {
        "messages": [
            { "role": "user", "content": "what are the top procedures performed?" }
        ]
    }
)
end_time = time.time()
print("Invoke time:", end_time-start_time)

Invoke time: 284.1119821071625


In [6]:
print(response)

{'messages': [HumanMessage(content='what are the top procedures performed?', additional_kwargs={}, response_metadata={}, id='bb2ea5be-c804-43d0-b5e3-606521ce3ee7'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-10-30T17:27:43.4432197Z', 'done': True, 'done_reason': 'stop', 'total_duration': 74471854800, 'load_duration': 28601221200, 'prompt_eval_count': 188, 'prompt_eval_duration': 11078476500, 'eval_count': 39, 'eval_duration': 23831766200, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--29a2aea6-94fa-4a1c-937b-c84dddaa2e32-0', tool_calls=[{'name': 'get_fhir_data', 'args': {'required': ['resourceType'], 'type': 'object'}, 'id': 'db914818-c60a-49bc-b6e7-2f0c80dcca86', 'type': 'tool_call'}], usage_metadata={'input_tokens': 188, 'output_tokens': 39, 'total_tokens': 227}), ToolMessage(content='[Document(metadata={\'source\': \'C:\\\\Users\\\\Peter\\\\Documents\\\\GitHub\\\\agentic-healthcare-analytics\\\\fh